In [111]:
import numpy as np
import pandas as pd
import os
import nltk
from gensim.models import Word2Vec
import re
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

In [112]:
df=pd.read_csv("post.csv")

In [113]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

In [114]:
def post_tokenizing(df):
    token_list=[]
    phrase_list=[]
    token_df=pd.DataFrame()
    token_df.insert(0,'Post',None)
    token_df.insert(1,'class',None)
    for val in df.values:
        append_list=[]
        filter_val=re.sub(r'Q:','',val[0])
        filter_val=re.sub(r'&#039;[a-z]{1}','',filter_val)
        filter_val=re.sub('<[a-z]+>',' ',filter_val).lower()
        value=re.sub(r'[^a-zA-Z\s]', '', filter_val, re.I|re.A)
        filter_tokens=[token for token in wpt.tokenize(value) if token not in stop_words and len(token)>=3]
        if(filter_tokens):
            join_words=' '.join(filter_tokens)
            append_list.append(join_words)
            append_list.append(val[1])
            token_df.loc[len(token_df)]=append_list
    return token_df

In [127]:
def post_vector_calculation(token_df):    
    texts=[]
    for val in token_df.values:
        texts.append(val[0])
    vec = TfidfVectorizer()
    tfidf = vec.fit_transform(texts)
    docs_vectors=pd.DataFrame(tfidf.toarray())
    docs_vectors['class']=token_df['class']
    return docs_vectors

In [128]:
def make_sample(df, number):
    sent_list=[]
    count=0
    for val in df.values:
        if(val[1]==1 and count!=number):
            sent_list.append(val)
            count=count+1
    count=0
    for val in df.values:
        if(val[1]==0 and count!=number):
            sent_list.append(val)
            count=count+1
    df_new=pd.DataFrame(sent_list)
    df_new.rename(columns={0:'Post',
                           1:'class'}, inplace=True)
    return df_new

In [129]:
token_df=post_tokenizing(df)
df_sample=make_sample(token_df,1000)
docs_vectors=post_vector_calculation(df_sample)
docs_vectors.head(5)

,0,1,2,3,4,5,6,7,8,9,...,5607,5608,5609,5610,5611,5612,5613,5614,5615,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [130]:
docs_vectors=docs_vectors.dropna()

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(docs_vectors.drop('class', axis = 1),docs_vectors['class'],test_size = 0.2,
                                                   random_state = 42)

In [132]:
print(X_train.shape, X_test.shape)

(1600, 5616) (400, 5616)


In [133]:
print(y_train.shape,y_test.shape)

(1600,) (400,)


In [134]:
model = AdaBoostClassifier(n_estimators=800, random_state = 42)
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, test_pred)

0.7475

In [135]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.model_selection import GridSearchCV
pipe = Pipeline(steps=[('svm', svm.SVC(probability=True))])
param_grid = {
    'svm__C': [0.1],  
    'svm__gamma': [1], 
    'svm__kernel': ['rbf']}
search = GridSearchCV(pipe, param_grid, cv=2, iid=False, refit=True)
search.fit(X_train, y_train)
print("Best CV score = %0.3f:" % search.best_score_)
print("Best parameters: ", search.best_params_)

SVM_best_params = search.best_params_
SVM_best_model = search.best_estimator_
print(SVM_best_model)
print(SVM_best_params)

Best CV score = 0.501:
Best parameters:  {'svm__C': 0.1, 'svm__gamma': 1, 'svm__kernel': 'rbf'}
Pipeline(memory=None,
         steps=[('svm',
                 SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=1,
                     kernel='rbf', max_iter=-1, probability=True,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)
{'svm__C': 0.1, 'svm__gamma': 1, 'svm__kernel': 'rbf'}


In [136]:
svm_test_predict=search.predict(X_test)

In [137]:
accuracy_score(y_test, svm_test_predict)

0.5

In [140]:
df_test=pd.read_csv("cleanprojectdataset.csv")
token_test_df=post_tokenizing(df_test)
docs_vectors_test=post_vector_calculation(token_test_df) 

In [141]:
print(docs_vectors_test.shape)

(1064, 3675)


In [142]:
test_pred_test=model.predict(docs_vectors_test.drop('class', axis = 1))
df_test_mod=token_test_df['class']
df_test_mod=df_test_mod.astype('int')
#df_test_mod=df_test_mod.drop(df_test_mod.index[65])
accuracy_score(df_test_mod, test_pred_test)

ValueError: Number of features of the model must match the input. Model n_features is 5616 and input n_features is 3674 

In [ ]:
test_pred_test=search.predict(docs_vector_test)
df_test_mod=df_test['class']
df_test_mod=df_test_mod.drop(df_test_mod.index[65])
accuracy_score(df_test_mod, test_pred_test)